# Tutorial

In order to run this notebook:
* create new environment, make it visible to your Jupyter
  * for conda do `conda create --name {name} python=3.10`
  * activate it and install `pip install ipykernel`
  * `ipython kernel install --user --name={name}`
* within the new environment, install requirements, e.g. `pip install -r requirements.txt`
  * this currently involves installing the current development versions of ms3 and dimcat
* clone the corpus: `git clone --recurse-submodules -j8 https://github.com/DCMLab/dcml_corpora`
* Set the `meta_repo` in the second cell to your local clone.

If the plots are not displayed and you are in JupyterLab, use [this guide](https://plotly.com/python/getting-started/#jupyterlab-support).

In [1]:
import os, random
from git import Repo
import dimcat as dc

In [2]:
meta_repo = "~/dcml_corpora"
repo = Repo(meta_repo)
print(f"{os.path.basename(meta_repo)} @ {repo.commit().hexsha[:7]}")
print(f"dimcat version {dc.__version__}")

dcml_corpora @ 4a7ad61
dimcat version 0.2.0.post1.dev64+gda0a036


## The Dataset object

A `Dataset` object represents one or several [DCML corpora](https://github.com/DCMLab/dcml_corpora), depending on which folder(s) you load. By default, DiMCAT will discover all data it can potentially load, but parse only the tabular TSV files, not the scores from which they have been derived.

### Initializing a Dataset

Pass a directory to `dimcat.Dataset.load()` to discover and parse all TSV files. The property `data` simply returns an `ms3.Parse` object.

In [3]:
dataset = dc.Dataset()
dataset.load(directory=meta_repo)
dataset.data

[default|all]
All corpora
-----------
View: This view is called 'default'. It 
	- excludes fnames that are not contained in the metadata,
	- filters out file extensions requiring conversion (such as .xml), and
	- excludes review files and folders.

                               has   active   scores measures           notes        expanded       
                          metadata     view detected detected parsed detected parsed detected parsed
corpus                                                                                              
ABC                            yes  default       70       70     70       70     70       70     70
beethoven_piano_sonatas        yes  default       87       87     87       87     87       64     64
chopin_mazurkas                yes  default       55       55     55       55     55       55     55
corelli                        yes  default      149      149    149      149    149      149    149
debussy_suite_bergamasque      yes  default 

### Data is organized in facets and IDs

* As you can see, DiMCAT recognized the subfolders of the meta-repository `dcml_corpora` as individual corpora (rows) and shows the number of detected and parsed files per facet (columns).
* The facets that a DCML dataset includes by default are
  * `measures`: feature matrices of partial and complete measure units that make up a score
  * `notes`:  feature matrices of all note heads contained in a score
  * `expanded`: feature matrices of all DCML harmony labels contained in a score
  * (`scores`): since the scores are considered to hold the latest version of the mentioned facets, DiMCAT can extract the information freshly, but for saving RAM it is recommended to simply keep your facets up to date using the `ms3 extract` command 
* Internally, DiMCAT addresses pieces using index tuples, also called indices or IDs, that take the form `('corpus_name', 'piece_name')`. The list of pieces currently being analysed is stored under the `.indices` field:

In [4]:
random.sample(dataset.indices, 10)

[('schumann_kinderszenen', 'n10'),
 ('tchaikovsky_seasons', 'op37a05'),
 ('beethoven_piano_sonatas', '16-2'),
 ('mozart_piano_sonatas', 'K282-2'),
 ('liszt_pelerinage', '160.05_Orage'),
 ('mozart_piano_sonatas', 'K281-3'),
 ('beethoven_piano_sonatas', '12-3'),
 ('medtner_tales', 'op42n01'),
 ('mozart_piano_sonatas', 'K280-3'),
 ('mozart_piano_sonatas', 'K309-2')]

### Accessing a facet
To inspect a facet concatenated over all pieces, simply call

In [5]:
dataset.get_facet('notes')

mc   mn quarterbeats  \
corpus              fname        interval                                
ABC                 n01op18-1_01 [0.0, 1.0)        1    1            0   
                                 [0.0, 1.0)        1    1            0   
                                 [0.0, 1.0)        1    1            0   
                                 [0.0, 1.0)        1    1            0   
                                 [1.0, 1.5)        1    1            1   
...                                              ...  ...          ...   
tchaikovsky_seasons op37a12      [525.0, 526.0)  176  176          525   
                                 [525.0, 526.0)  176  176          525   
                                 [525.0, 526.0)  176  176          525   
                                 [525.0, 526.0)  176  176          525   
                                 [525.0, 526.0)  176  176          525   

                                                 duration_qb mc_onset  \
corpus              fname        interval                               
ABC                 n01op18-1_01 [0.0, 1.0)              1.0        0   
                                 [0.0, 1.0)              1.0        0   
                                 [0.0, 1.0)              1.0        0   
                                 [0.0, 1.0)              1.0        0   
                                 [1.0, 1.5)              0.5      1/4   
...                                                      ...      ...   
tchaikovsky_seasons op37a12      [525.0, 526.0)          1.0        0   
                                 [525.0, 526.0)          1.0        0   
                                 [525.0, 526.0)          1.0        0   
                                 [525.0, 526.0)          1.0        0   
                                 [525.0, 526.0)          1.0        0   

                                                mn_onset timesig  staff  \
corpus              fname        interval                                 
ABC                 n01op18-1_01 [0.0, 1.0)            0     3/4      3   
                                 [0.0, 1.0)            0     3/4      4   
                                 [0.0, 1.0)            0     3/4      1   
                                 [0.0, 1.0)            0     3/4      2   
                                 [1.0, 1.5)          1/4     3/4      3   
...                                                  ...     ...    ...   
tchaikovsky_seasons op37a12      [525.0, 526.0)        0     3/4      2   
                                 [525.0, 526.0)        0     3/4      2   
                                 [525.0, 526.0)        0     3/4      1   
                                 [525.0, 526.0)        0     3/4      1   
                                 [525.0, 526.0)        0     3/4      1   

                                                 voice duration  ...  \
corpus              fname        interval                        ...   
ABC                 n01op18-1_01 [0.0, 1.0)          1      1/4  ...   
                                 [0.0, 1.0)          1      1/4  ...   
                                 [0.0, 1.0)          1      1/4  ...   
                                 [0.0, 1.0)          1      1/4  ...   
                                 [1.0, 1.5)          1      1/8  ...   
...                                                ...      ...  ...   
tchaikovsky_seasons op37a12      [525.0, 526.0)      1      1/4  ...   
                                 [525.0, 526.0)      1      1/4  ...   
                                 [525.0, 526.0)      1      1/4  ...   
                                 [525.0, 526.0)      1      1/4  ...   
                                 [525.0, 526.0)      1      1/4  ...   

                                                nominal_duration scalar  tied  \
corpus              fname        interval                                       
ABC                 n01op18-1_01 [0.0, 1.0)                  1

As you can see, the IDs are included as index levels and the third level indicates the temporal dimensions, the time interval, of each event (here of every note head) expressed in quarter notes.

### Filtering the data to be analyzed

The data overview above shows that the `expanded` facet is not available for all 87 pieces included in the `beethoven_piano_sonatas` corpus (i.e., they have not been annotated with harmony labels), so in some cases we might want to filter pieces (i.e., IDs) from the dataset. This is what the objects in the `dimcat.filter` module do: They iterate through the IDs and exclude those that do not fulfill the filter's criterion.

For example, to filter out those pieces that have not been annotated we instantiate an `IsAnnotatedFilter`:

In [6]:
filter_object = dc.IsAnnotatedFilter()

All DiMCAT objects working on data come with a `.process_data()` method that takes a dataset and returns a processed copy of the dataset. For example, to apply the filter to our dataset:

In [7]:
annotated = filter_object.process_data(dataset)
print(f"Before filtering: {dataset.n_indices} IDs.\nAfter filtering: {annotated.n_indices} IDs.")

INCOMPLETE_MC_WRONGLY_COMPLETED_WARNING (3, 96) ms3.Parse.beethoven_piano_sonatas.04-3 -- /home/hentsche/PycharmProjects/ms3/src/ms3/bs4_measures.py (line 763) make_offset_col():
	The incomplete MC 96 (timesig 3/4, act_dur 1/4) is completed by 1 incorrect duration (expected: 1/2):
	{97: Fraction(3, 4)}
INCOMPLETE_MC_WRONGLY_COMPLETED_WARNING (3, 112) ms3.Parse.beethoven_piano_sonatas.04-3 -- /home/hentsche/PycharmProjects/ms3/src/ms3/bs4_measures.py (line 763) make_offset_col():
	The incomplete MC 112 (timesig 3/4, act_dur 1/2) is completed by 1 incorrect duration (expected: 1/4):
	{25: Fraction(3, 4), 113: Fraction(1, 4)}
INCOMPLETE_MC_WRONGLY_COMPLETED_WARNING (3, 39) ms3.Parse.beethoven_piano_sonatas.13-1 -- /home/hentsche/PycharmProjects/ms3/src/ms3/bs4_measures.py (line 763) make_offset_col():
	The incomplete MC 39 (timesig 3/4, act_dur 1/8) is completed by 1 incorrect duration (expected: 5/8):
	{40: Fraction(3, 4)}
INCOMPLETE_MC_WRONGLY_COMPLETED_WARNING (3, 47) ms3.Parse.beethov

## Applying PipelineSteps to a Dataset

Everything else in DiMCAT is a PipelineStep the various types of which are distributed over several modules:

* `dimcat.filter`: Filters return a new Dataset where certain IDs have been removed.
* `dimcat.plotter`: Plotters plot analysis ('processed') data and potentially output plots as files.
* `dimcat.writer`: Writers output analyzed data to disk.

* `dimcat.grouper`: Groupers subdivide each of the current ID groups based on a given criterion and return a new Dataset with an altered `.indices` field.
* `dimcat.slicer`: Slicers create for each ID (read: piece) a set of chunks identified by non-overlapping intervals. Any facet retrieved from such a sliced Dataset will be sliced, cutting and duplicating any event that overlaps the interval boundaries.
* `dimcat.analyzer`: Analyzers perform an analysis on a given Dataset and return a new Dataset with the results stored in the `.processed` field. 

As we have seen with the `IsAnnotatedFilter` above, these PipelineSteps come with the method `process_data()` and return a copy of the given Dataset. Out of the first three mentioned types, it is actually the only one returning a potentially modified copy, whereas plotters and writers return an exact copy.

This is different for groupers, slicers, and analyzers which all transform datasets to the point that they output new datatypes, which are subtypes of `GroupedData`, `SlicedData`, and `AnalyzedData` respectively.

### Applying a slicer

Having selected only annotated pieces, we can apply the `LocalKeySlicer` that slices the dataset into segments that remaining in one and the same local key:

In [8]:
localkey_slices = dc.LocalKeySlicer().process_data(annotated)
type(localkey_slices)

dimcat.data.SlicedDataset

The type of the new dataset is a hybrid of `SlicedData` (any data processed by a slicer) and `Dataset`, the particular type representing for DCML corpora:

In [9]:
isinstance(localkey_slices, dc.data.SlicedData), isinstance(localkey_slices, dc.Dataset)

(True, True)

This is useful because we can guarantee that sliced data always come with its two defining fields, which are the dictionary `sliced` for storing previously sliced facets, and `.slice_info` for storing information about the slices themselves; regardless of the type of the dataset and its idiosyncratic ways of interacting with this slicing information.

Another thing differentiating `SlicedData` from other data is that it has new IDs which identify pieces rather than slices:

In [10]:
print(f"Before: {annotated.n_indices} IDs, after slicing: {localkey_slices.n_indices} IDs")
print(f"Facets that have been sliced so far: {list(localkey_slices.sliced.keys())}.")

Before: 537 IDs, after slicing: 3468 IDs
Facets that have been sliced so far: ['expanded'].


The IDs of the sliced Dataset have multiplied and received a third element, which is the interval specifying the extent of one particular slice in quarter notes. Let's have a look at the first 10 IDs of the `SlicedDataset`:

In [11]:
localkey_slices.indices[:10]

[('ABC', 'n01op18-1_01', Interval(0.0, 117.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(117.0, 163.5, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(163.5, 342.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(342.0, 354.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(354.0, 378.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(378.0, 405.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(405.0, 423.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(423.0, 438.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(438.0, 444.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(444.0, 474.0, closed='left'))]

The IDs make sure that all facets retrieved from this Dataset will be sliced.

This is True not only for the facet that has been used for slicing (annotation tables in the present case):

In [12]:
localkey_slices.get_facet('expanded').head(30)

mc  mn quarterbeats  \
corpus fname        localkey_slice interval                            
ABC    n01op18-1_01 [0.0, 117.0)   [0.0, 3.0)     1   1            0   
                                   [3.0, 6.0)     2   2            3   
                                   [6.0, 9.0)     3   3            6   
                                   [9.0, 15.0)    4   4            9   
                                   [15.0, 18.0)   6   6           15   
                                   [18.0, 19.0)   7   7           18   
                                   [19.0, 20.0)   7   7           19   
                                   [20.0, 21.0)   7   7           20   
                                   [21.0, 23.0)   8   8           21   
                                   [23.0, 24.0)   8   8           23   
                                   [24.0, 27.0)   9   9           24   
                                   [27.0, 30.0)  10  10           27   
                                   [30.0, 33.0)  11  11           30   
                                   [33.0, 39.0)  12  12           33   
                                   [39.0, 41.0)  14  14           39   
                                   [41.0, 45.0)  14  14           41   
                                   [45.0, 47.0)  16  16           45   
                                   [47.0, 51.0)  16  16           47   
                                   [51.0, 52.5)  18  18           51   
                                   [52.5, 54.0)  18  18        105/2   
                                   [54.0, 56.0)  19  19           54   
                                   [56.0, 56.5)  19  19           56   
                                   [56.5, 57.0)  19  19        113/2   
                                   [57.0, 60.0)  20  20           57   
                                   [60.0, 63.0)  21  21           60   
                                   [63.0, 65.0)  22  22           63   
                                   [65.0, 69.0)  22  22           65   
                                   [69.0, 71.0)  24  24           69   
                                   [71.0, 75.0)  24  24           71   
                                   [75.0, 77.0)  26  26           75   

                                                 duration_qb mc_onset  \
corpus fname        localkey_slice interval                             
ABC    n01op18-1_01 [0.0, 117.0)   [0.0, 3.0)            3.0        0   
                                   [3.0, 6.0)            3.0        0   
                                   [6.0, 9.0)            3.0        0   
                                   [9.0, 15.0)           6.0        0   
                                   [15.0, 18.0)          3.0        0   
                                   [18.0, 19.0)          1.0        0   
                                   [19.0, 20.0)          1.0      1/4   
                                   [20.0, 21.0)          1.0      1/2   
                                   [21.0, 23.0)          2.0        0   
                                   [23.0, 24.0)          1.0      1/2   
                                   [24.0, 27.0)          3.0        0   
                                   [27.0, 30.0)          3.0        0   
                                   [30.0, 33.0)          3.0        0   
                                   [33.0, 39.0)          6.0        0   
                                   [39.0, 41.0)          2.0        0   
                                   [41.0, 45.0)          4.0      1/2   
                                   [45.0, 47.0)          2.0        0   
                                   [47.0, 51.0)          4.0      1/2   
                                   [51.0, 52.5)          1.5        0   
                                   [52.5, 54.0)          1.5      3/8   
                                   [54.0, 56.0)          2.0        0   
                                   [56.0, 56.5)          0.5     

But also for any other facet requested:

In [13]:
localkey_slices.get_facet('notes')

mc   mn  \
corpus              fname        localkey_slice                            
ABC                 n01op18-1_01 [0.0, 117.0)   [0.0, 1.0)        1    1   
                                                [0.0, 1.0)        1    1   
                                                [0.0, 1.0)        1    1   
                                                [0.0, 1.0)        1    1   
                                                [1.0, 1.5)        1    1   
...                                                             ...  ...   
tchaikovsky_seasons op37a12      [442.0, 528.0) [525.0, 526.0)  176  176   
                                                [525.0, 526.0)  176  176   
                                                [525.0, 526.0)  176  176   
                                                [525.0, 526.0)  176  176   
                                                [525.0, 526.0)  176  176   

                                                               quarterbeats  \
corpus              fname        localkey_slice                               
ABC                 n01op18-1_01 [0.0, 117.0)   [0.0, 1.0)                0   
                                                [0.0, 1.0)                0   
                                                [0.0, 1.0)                0   
                                                [0.0, 1.0)                0   
                                                [1.0, 1.5)                1   
...                                                                     ...   
tchaikovsky_seasons op37a12      [442.0, 528.0) [525.0, 526.0)          525   
                                                [525.0, 526.0)          525   
                                                [525.0, 526.0)          525   
                                                [525.0, 526.0)          525   
                                                [525.0, 526.0)          525   

                                                                duration_qb  \
corpus              fname        localkey_slice                               
ABC                 n01op18-1_01 [0.0, 117.0)   [0.0, 1.0)              1.0   
                                                [0.0, 1.0)              1.0   
                                                [0.0, 1.0)              1.0   
                                                [0.0, 1.0)              1.0   
                                                [1.0, 1.5)              0.5   
...                                                                     ...   
tchaikovsky_seasons op37a12      [442.0, 528.0) [525.0, 526.0)          1.0   
                                                [525.0, 526.0)          1.0   
                                                [525.0, 526.0)          1.0   
                                                [525.0, 526.0)          1.0   
                                                [525.0, 526.0)          1.0   

                                                               mc_onset  \
corpus              fname        localkey_slice                           
ABC                 n01op18-1_01 [0.0, 117.0)   [0.0, 1.0)            0   
                                                [0.0, 1.0)            0   
                                                [0.0, 1.0)            0   
                                                [0.0, 1.0)            0   
                                                [1.0, 1.5)          1/4   
...                                                                 ...   
tchaikovsky_seasons op37a12      [442.0, 528.0) [525.0, 526.0)        0   
                                                [525.0, 526.0)        0   
                                                [525.0, 526.0)        0   
                                                [525.0, 526.0)        0   
                                                [525.0, 526.0)        0   

                                            

In both cases we see an additional index level `localkey_slice` containing the intervals of the localkey segments. Notes that originally overlapped a localkey boundary are now split in two with `duration_qb` values adapted (but not `duration` which keeps the original value). 

However, we might be interested only in the slices themselves, so we can get the information stored in the field `slice_info` by calling:

In [14]:
localkey_slices.get_slice_info()[['duration_qb', 'globalkey', 'localkey']]

duration_qb globalkey localkey
corpus              fname        localkey_slice                               
ABC                 n01op18-1_01 [0.0, 117.0)         117.0         F        I
                                 [117.0, 163.5)        46.5         F        v
                                 [163.5, 342.0)       178.5         F        V
                                 [342.0, 354.0)        12.0         F      III
                                 [354.0, 378.0)        24.0         F       IV
...                                                     ...       ...      ...
tchaikovsky_seasons op37a12      [261.0, 303.0)        42.0        Ab      bVI
                                 [303.0, 309.0)         6.0        Ab  iii/bVI
                                 [309.0, 399.0)        90.0        Ab      bVI
                                 [399.0, 442.0)        43.0        Ab  iii/bVI
                                 [442.0, 528.0)        86.0        Ab        I

[3468 rows x 3 columns]

### Applying a Grouper

If, for example, we want to analyse localkey segments separately depending on whether they are in major or minor, we could apply a `ModeGrouper`, which can only applied to a Dataset that has already been sliced:

In [15]:
grouped_localkey_slices = dc.ModeGrouper().process_data(localkey_slices)
grouped_localkey_slices.get_slice_info()[['duration_qb', 'globalkey', 'localkey']]

duration_qb  \
localkey_is_minor corpus              fname        localkey_slice               
False             ABC                 n01op18-1_01 [0.0, 117.0)         117.0   
                                                   [163.5, 342.0)       178.5   
                                                   [342.0, 354.0)        12.0   
                                                   [354.0, 378.0)        24.0   
                                                   [498.0, 564.0)        66.0   
...                                                                       ...   
True              tchaikovsky_seasons op37a11      [155.0, 166.0)        11.0   
                                                   [171.0, 180.0)         9.0   
                                                   [224.0, 232.0)         8.0   
                                      op37a12      [303.0, 309.0)         6.0   
                                                   [399.0, 442.0)        43.0   

                                                                  globalkey  \
localkey_is_minor corpus              fname        localkey_slice             
False             ABC                 n01op18-1_01 [0.0, 117.0)           F   
                                                   [163.5, 342.0)         F   
                                                   [342.0, 354.0)         F   
                                                   [354.0, 378.0)         F   
                                                   [498.0, 564.0)         F   
...                                                                     ...   
True              tchaikovsky_seasons op37a11      [155.0, 166.0)         E   
                                                   [171.0, 180.0)         E   
                                                   [224.0, 232.0)         E   
                                      op37a12      [303.0, 309.0)        Ab   
                                                   [399.0, 442.0)        Ab   

                                                                  localkey  
localkey_is_minor corpus              fname        localkey_slice           
False             ABC                 n01op18-1_01 [0.0, 117.0)          I  
                                                   [163.5, 342.0)        V  
                                                   [342.0, 354.0)      III  
                                                   [354.0, 378.0)       IV  
                                                   [498.0, 564.0)        I  
...                                                                    ...  
True              tchaikovsky_seasons op37a11      [155.0, 166.0)       iv  
                                                   [171.0, 180.0)        v  
                                                   [224.0, 232.0)      iii  
                                      op37a12      [303.0, 309.0)  iii/bVI  
                                                   [399.0, 442.0)  iii/bVI  

[3468 rows x 3 columns]

The grouping is displayed as the prepended index level `localkey_is_minor`. In this case the groups are simply called `True` or `False`, as can be seen by inspecting the `.indices` dictionary. The keys are tuples whose lengths match the number of applied Groupers so far.

In [16]:
list(grouped_localkey_slices.indices)

[('ABC', 'n01op18-1_01', Interval(0.0, 117.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(117.0, 163.5, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(163.5, 342.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(342.0, 354.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(354.0, 378.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(378.0, 405.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(405.0, 423.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(423.0, 438.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(438.0, 444.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(444.0, 474.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(474.0, 498.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(498.0, 564.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(564.0, 591.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(591.0, 618.0, closed='left')),
 ('ABC', 'n01op18-1_01', Interval(618.0, 645.5, closed='left')),
 ('ABC', 'n01op18-1_01', In

## Applying an Analyzer

After having seen the various ways how a Dataset can be reshaped, let us have a look how the various transformations change the result of an analyzer.
To that aim, let's first initialize the `PitchClassVectors` analyzer with the desired configuration:

In [17]:
pcv_analyzer = dc.PitchClassVectors(pitch_class_format='pc', 
                                    weight_grace_durations=0.5, 
                                    normalize=True, 
                                    include_empty=True)

We want to 

* see pitch classes 0-12 (as opposed to the defautl `tpc`, i.e. tonal pitch classes on the line of fifth),
* include grace notes, which usually have duration 0, by halving their note values,
* normalize the resulting vectors, and
* include zero vectors where no notes occur (i.e. for completely silent segments).

We start by applying this analyzer to the filtered dataset, in which all pieces are excluded that do not contain annotations:

In [18]:
annotated_pieces_pcvs = pcv_analyzer.process_data(annotated)
annotated_pieces_pcvs.get_results()

0         1         2   \
corpus                fname                                            
ABC                   n01op18-1_01      0.184561  0.055635  0.070421   
                      n01op18-1_02      0.080528  0.066491  0.150705   
                      n01op18-1_03      0.215696  0.060841  0.106149   
                      n01op18-1_04      0.183797  0.036084  0.085030   
                      n02op18-2_01      0.072496  0.040254  0.195718   
...                                          ...       ...       ...   
grieg_lyrical_pieces  op47n04                NaN  0.025747  0.385170   
                      op62n03                NaN  0.097554  0.107645   
                      op65n02                NaN  0.146921  0.095015   
liszt_pelerinage      160.03_Pastorale       NaN  0.065248  0.005674   
schumann_kinderszenen n10                    NaN  0.076590  0.008671   

                                              3         4         5   \
corpus                fname                                            
ABC                   n01op18-1_01      0.042324  0.084489  0.137893   
                      n01op18-1_02      0.024917  0.097261  0.114579   
                      n01op18-1_03      0.030421  0.079207  0.122087   
                      n01op18-1_04      0.027207  0.085792  0.142276   
                      n02op18-2_01      0.034508  0.088217  0.032877   
...                                          ...       ...       ...   
grieg_lyrical_pieces  op47n04                NaN  0.038105       NaN   
                      op62n03           0.037920  0.247401       NaN   
                      op65n02           0.028152  0.171261  0.023460   
liszt_pelerinage      160.03_Pastorale  0.051064  0.320922       NaN   
schumann_kinderszenen n10               0.229769  0.063584  0.023121   

                                              6         7         8   \
corpus                fname                                            
ABC                   n01op18-1_01      0.029651  0.128686  0.044994   
                      n01op18-1_02      0.039483  0.107511  0.019767   
                      n01op18-1_03      0.017152  0.123301  0.056958   
                      n01op18-1_04      0.027104  0.151199  0.041352   
                      n02op18-2_01      0.085095  0.130405  0.019561   
...                                          ...       ...       ...   
grieg_lyrical_pieces  op47n04           0.024717  0.012358  0.041195   
                      op62n03           0.067584  0.015291  0.112538   
                      op65n02           0.089150  0.018768  0.100880   
liszt_pelerinage      160.03_Pastorale  0.103901       NaN  0.081915   
schumann_kinderszenen n10               0.104046  0.060694  0.122832   

                                              9         10        11  
corpus                fname                                           
ABC                   n01op18-1_01      0.081221  0.099793  0.040332  
                      n01op18-1_02      0.182614  0.087874  0.028271  
                      n01op18-1_03      0.072411  0.080825  0.034951  
                      n01op18-1_04      0.108558  0.077516  0.034085  
                      n02op18-2_01      0.128167  0.041636  0.131068  
...                                          ...       ...       ...  
grieg_lyrical_pieces  op47n04           0.447992  0.004119  0.020597  
                      op62n03           0.174924  0.007339  0.131804  
                      op65n02           0.214956  0.009384  0.102053  
liszt_pelerinage      160.03_Pastorale  0.072695  0.005319  0.293262  
schumann_kinderszenen n10                    NaN  0.109827  0.200867  

[537 rows x 12 columns]

Applying the same analyzer to the Dataset sliced by localkey segments yields one vector per segment:

In [19]:
localkey_segment_pcvs = pcv_analyzer.process_data(localkey_slices)
localkey_segment_pcvs.get_results()

0         1   \
corpus               fname        localkey_slice                       
ABC                  n01op18-1_01 [0.0, 117.0)    0.198730  0.018413   
                                  [117.0, 163.5)  0.171348  0.042135   
                                  [163.5, 342.0)  0.207028  0.005819   
                                  [354.0, 378.0)  0.081433  0.006515   
                                  [378.0, 405.0)  0.014085  0.154930   
...                                                    ...       ...   
mozart_piano_sonatas K331-3       [137.0, 144.0)       NaN       NaN   
tchaikovsky_seasons  op37a02      [34.0, 38.0)         NaN       NaN   
                                  [150.0, 154.0)       NaN       NaN   
                                  [294.0, 298.0)       NaN       NaN   
                     op37a12      [303.0, 309.0)       NaN       NaN   

                                                        2         3   \
corpus               fname        localkey_slice                       
ABC                  n01op18-1_01 [0.0, 117.0)    0.083175  0.025397   
                                  [117.0, 163.5)  0.103933  0.181180   
                                  [163.5, 342.0)  0.118621  0.021486   
                                  [354.0, 378.0)  0.045603  0.133550   
                                  [378.0, 405.0)  0.084507  0.045775   
...                                                    ...       ...   
mozart_piano_sonatas K331-3       [137.0, 144.0)       NaN  0.022346   
tchaikovsky_seasons  op37a02      [34.0, 38.0)         NaN  0.097561   
                                  [150.0, 154.0)       NaN  0.097561   
                                  [294.0, 298.0)       NaN  0.116279   
                     op37a12      [303.0, 309.0)       NaN  0.281250   

                                                        4         5   \
corpus               fname        localkey_slice                       
ABC                  n01op18-1_01 [0.0, 117.0)    0.081905  0.207937   
                                  [117.0, 163.5)  0.018258  0.042135   
                                  [163.5, 342.0)  0.132274  0.102059   
                                  [354.0, 378.0)  0.003257  0.081433   
                                  [378.0, 405.0)  0.204225  0.066901   
...                                                    ...       ...   
mozart_piano_sonatas K331-3       [137.0, 144.0)  0.379888       NaN   
tchaikovsky_seasons  op37a02      [34.0, 38.0)    0.097561       NaN   
                                  [150.0, 154.0)  0.097561       NaN   
                                  [294.0, 298.0)  0.116279       NaN   
                     op37a12      [303.0, 309.0)       NaN       NaN   

                                                        6         7   \
corpus               fname        localkey_slice                       
ABC                  n01op18-1_01 [0.0, 117.0)    0.020317  0.119683   
                                  [117.0, 163.5)  0.053371  0.168539   
                                  [163.5, 342.0)  0.022829  0.209490   
                                  [354.0, 378.0)       NaN  0.159609   
                                  [378.0, 405.0)       NaN  0.176056   
...                                                    ...       ...   
mozart_piano_sonatas K331-3       [137.0, 144.0)  0.134078  0.075419   
tchaikovsky_seasons  op37a02      [34.0, 38.0)    0.121951  0.195122   
                                  [150.0, 154.0)  0.121951  0.195122   
                                  [294.0, 298.0)  0.116279  0.186047   
                     op37a12      [303.0, 309.0)  0.125000  0.093750   

                                                        8         9   \
corpus               fname        localkey_slice                       
ABC                  n01op18-1_01 [0.0, 117.0)    0.007619  0.111111   
                                  [117.0, 163.5)  0.058989  0.036517   
                    

Applying a `PitchClassVectors` Analyzer to the localkey segments that have been grouped by keys, seems to not make much of a difference
(except that this one here does not normalize):

In [20]:
grouped_localkey_pcvs = dc.PitchClassVectors(pitch_class_format='pc').process_data(grouped_localkey_slices)
grouped_localkey_pcvs.get_group_results()

,0,1,2,3,4,5,6,7,8,9,10,11
localkey_is_minor,,,,,,,,,,,,
False,38232.886712,21674.846314,39868.293621,25266.823683,35823.829104,32468.277940,23832.922872,40380.157915,23658.902808,37866.390984,26619.865639,29551.462858
True,21862.261310,12444.749603,19534.402183,11675.217163,20465.950893,16199.250099,13081.539583,21818.557440,13260.370536,18478.893552,11426.841567,16627.950794


In [21]:
grouped_localkey_pcvs.get_results()

0      1      2      3   \
corpus               fname        localkey_slice                                
ABC                  n01op18-1_01 [0.0, 117.0)     78.25   7.25  32.75  10.00   
                                  [117.0, 163.5)   30.50   7.50  18.50  32.25   
                                  [163.5, 342.0)  115.50   3.25  66.00  12.00   
                                  [354.0, 378.0)    6.25   0.50   3.50  10.25   
                                  [378.0, 405.0)    1.00  11.00   6.00   3.25   
...                                                  ...    ...    ...    ...   
mozart_piano_sonatas K331-3       [137.0, 144.0)     NaN    NaN    NaN   0.50   
tchaikovsky_seasons  op37a02      [34.0, 38.0)       NaN    NaN    NaN   2.00   
                                  [150.0, 154.0)     NaN    NaN    NaN   2.00   
                                  [294.0, 298.0)     NaN    NaN    NaN   2.50   
                     op37a12      [303.0, 309.0)     NaN    NaN    NaN   4.50   

                                                     4      5      6       7   \
corpus               fname        localkey_slice                                
ABC                  n01op18-1_01 [0.0, 117.0)    32.00  81.50   8.00   46.75   
                                  [117.0, 163.5)   3.25   7.50   9.50   30.00   
                                  [163.5, 342.0)  73.75  57.00  12.75  117.00   
                                  [354.0, 378.0)   0.25   6.25    NaN   12.25   
                                  [378.0, 405.0)  14.50   4.75    NaN   12.50   
...                                                 ...    ...    ...     ...   
mozart_piano_sonatas K331-3       [137.0, 144.0)   8.50    NaN   3.00    1.50   
tchaikovsky_seasons  op37a02      [34.0, 38.0)     2.00    NaN   2.50    4.00   
                                  [150.0, 154.0)   2.00    NaN   2.50    4.00   
                                  [294.0, 298.0)   2.50    NaN   2.50    4.00   
                     op37a12      [303.0, 309.0)    NaN    NaN   2.00    1.50   

                                                    8      9      10     11  
corpus               fname        localkey_slice                             
ABC                  n01op18-1_01 [0.0, 117.0)     3.0  43.25  38.75  10.50  
                                  [117.0, 163.5)  10.5   6.50   8.75  13.25  
                                  [163.5, 342.0)  20.0  27.50   9.00  44.00  
                                  [354.0, 378.0)   NaN   5.50  32.00    NaN  
                                  [378.0, 405.0)   NaN   4.00  14.00    NaN  
...                                                ...    ...    ...    ...  
mozart_piano_sonatas K331-3       [137.0, 144.0)   NaN   2.50    NaN   6.00  
tchaikovsky_seasons  op37a02      [34.0, 38.0)     NaN   2.00    NaN   8.00  
                                  [150.0, 154.0)   NaN   2.00    NaN   8.00  
                                  [294.0, 298.0)   NaN   2.00    NaN   8.00  
                     op37a12      [303.0, 309.0)   3.0    NaN   3.50   1.50  

[3468 rows x 12 columns]

However, the previous grouping allows us to iterate through the grouped pitch class vectors, e.g. for summing them up for all segments in major and minor respectively:

In [22]:
for (mode,), pcvs in grouped_localkey_pcvs.iter_group_results():
    print(f"PITCH CLASS PROFILE FOR ALL {'MINOR' if mode else 'MAJOR'} SEGMENTS:")
    display(pcvs / pcvs.sum())

PITCH CLASS PROFILE FOR ALL MAJOR SEGMENTS:


pc
0     0.101888
1     0.057762
2     0.106246
3     0.067334
4     0.095468
5     0.086526
6     0.063513
7     0.107610
8     0.063049
9     0.100911
10    0.070940
11    0.078753
Name: duration_qb, dtype: float64

PITCH CLASS PROFILE FOR ALL MINOR SEGMENTS:


pc
0     0.111046
1     0.063211
2     0.099222
3     0.059302
4     0.103954
5     0.082281
6     0.066446
7     0.110824
8     0.067354
9     0.093861
10    0.058041
11    0.084459
Name: duration_qb, dtype: float64

### Analyzing slice infos

In [23]:
lk_per_piece = dc.PieceGrouper().process_data(localkey_slices)
lokalkeys_per_piece = dc.LocalKeySequence().process_data(lk_per_piece)
lokalkeys_per_piece.get_results()

ValueError: Length of names must match number of levels in MultiIndex.

In [ ]:
unique_localkeys = dc.LocalKeyUnique().process_data(lk_per_piece)
unique_localkeys.get_results()